In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

import matplotlib.pyplot as plt
from PIL import Image
import random, os
import numpy as np
import shap

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#train and test data directory
data_dir = "/kaggle/input/pcos-detection-using-ultrasound-images/data/train"
test_data_dir = "/kaggle/input/pcos-detection-using-ultrasound-images/data/test"

In [ ]:


class ImageFolderEX(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            sample = self.loader(path)
        except:
            return None
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

In [ ]:
transform = transforms.Compose([transforms.Resize((256,256)),
                                transforms.ToTensor()])

#load the train and test data
dataset = ImageFolderEX(data_dir, transform = transform)
test_ds = ImageFolderEX(test_data_dir, transform = transform)

print('dataset samples:', len(dataset))
print('test samples:', len(test_ds))
print("Classes are:", dataset.classes)

img, label = dataset[0]
print(f'Example: first sample size: {img.shape}, and label: {label}')

In [ ]:
batch_size = 128
val_size = 400
train_size = len(dataset) - val_size

train_data, val_data = random_split(dataset, [train_size, val_size])

print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

In [ ]:

def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    return torch.utils.data.dataloader.default_collate(batch)

train_dl = DataLoader(train_data, batch_size, shuffle = True, collate_fn=collate_fn, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, collate_fn=collate_fn, num_workers = 4, pin_memory = True)
test_dl = DataLoader(test_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)

In [ ]:

images, labels = next(iter(train_dl))
print(images.shape , labels.shape)

plt.figure(figsize=(8,4))

for i in range(8):
    plt.subplot(2,4,i+1)
    plt.imshow(images[i,0], cmap=plt.cm.gray)
    if labels[i]==0:
        plt.title('Infected', fontsize=10)
    else:
        plt.title('Not Infected', fontsize=10)
    plt.axis("off")

In [ ]:
class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        images = images.to(device)
        labels = labels.to(device)
        
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [ ]:
class Classifier(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 12, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(3,3),
            
            nn.Conv2d(12, 15, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(3,3),
        
            nn.Conv2d(15, 10, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(3,3),
            
            nn.Flatten(),
            nn.Linear(810, 2),
        )
    
    def forward(self, xb):
        return self.network(xb)

In [ ]:
model = Classifier().to(device)

# for name, params in model.named_parameters():
#     print('layer name: {}, \t params shape: {}'.format(name, params.shape))

In [ ]:
num_epochs = 20
opt_func = torch.optim.Adam
lr = 0.001

#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

In [ ]:
#save the model
torch.save(model.state_dict(), 'PCOS_detection_{}_epochs_val_acc_{}.pth'.format(num_epochs, history[-1]['val_acc']))

In [ ]:
def plot_accuracies(history):
    """ Plot the history of accuracies"""
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Validation accuracy vs. No. of epochs');

plot_accuracies(history)

In [ ]:
def plot_losses(history):
    """ Plot the losses in each epoch"""
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

plot_losses(history)

## Evaluate on Test Dataset

In [ ]:
result = evaluate(model, test_dl)
result

In [ ]:
def predict_img_class(img,model):
    """ Predict the class of image and Return Predicted Class"""
    img = img.unsqueeze(0).to(device)
    prediction =  model(img)
    _, preds = torch.max(prediction, dim = 1)
    return dataset.classes[preds[0].item()]

In [ ]:
test_img = random.choice(os.listdir('/kaggle/input/pcos-detection-using-ultrasound-images/data/test/infected'))
img = Image.open(f'/kaggle/input/pcos-detection-using-ultrasound-images/data/test/infected/{test_img}')
img = transform(img)
plt.imshow(img.permute(1,2,0))

pred = predict_img_class(img,model)
print(f"Predicted Class: {pred}")

In [ ]:
# Use CPU for SHAP explainer
shap_device = torch.device('cpu')
model = model.to(shap_device)

In [ ]:
#Load a batch from train data for background
background, labels = next(iter(train_dl))
background = background.to(shap_device)

#Load a batch from test data
test_images, test_labels = next(iter(test_dl))

print('background images:', background.shape)
print('background labels:', labels.shape)
print('test images:', test_images.shape)

In [ ]:
#Create the SHAP explainer and get shap values
explainer = shap.DeepExplainer(model, background)
shap_values = explainer.shap_values(test_images)

print('Shap values:', len(shap_values), shap_values[0].shape)

In [ ]:
# Convert to numpy and reshape for plotting
shap_numpy = list(np.array(shap_values).transpose(0,1,3,4,2))
test_numpy = np.array(test_images).transpose(0,2,3,1)

print('Shap values, numpy:', len(shap_numpy), shap_numpy[0].shape)
print('test images, numpy:', test_numpy.shape)

In [ ]:
test_list_labels = list(np.array(test_labels))
GT_labels = [dataset.classes[test_list_labels[i].item()] for i in range(len(test_list_labels))]
print(GT_labels)

In [ ]:
# shap.image_plot(shap_numpy, test_numpy, labels=dataset.classes, true_labels=GT_labels)
shap.image_plot(shap_numpy, test_numpy, true_labels=GT_labels)